In [1]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

In [2]:
spark=SparkSession.builder\
    .appName("Scaler EDA").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/21 03:22:14 INFO SparkEnv: Registering MapOutputTracker
24/07/21 03:22:14 INFO SparkEnv: Registering BlockManagerMaster
24/07/21 03:22:14 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/07/21 03:22:14 INFO SparkEnv: Registering OutputCommitCoordinator


In [9]:
df=spark.read.csv(path="/user/sovik/target_data.csv",header=True,inferSchema=True)

In [48]:
df.show()

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date|      customer_city|customer_state|customer_zip_code_prefix|review_score|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+
|  1|   delivered|                79.0|               17.8|              1|     2017-10-02 10:56:00|2017-10-02 11:07:00|          2017-10-10 21:25:00|           Luziania|            GO|                     728|           5|
|  2|   delivered|               119.9|              27.16|              1|     2018-07-24 20:41:00|2018

In [10]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- review_score: integer (nullable = true)



Calculate the Mean of “order_products_value” , ”order_freight_value”

In [17]:
df.select(round(mean(col("order_products_value")), 2).alias("order_product_value_mean")).show()

+------------------------+
|order_product_value_mean|
+------------------------+
|                  127.88|
+------------------------+



In [18]:
df.select(round(mean(col("order_freight_value")), 2).alias("order_freight_value_mean")).show()

+------------------------+
|order_freight_value_mean|
+------------------------+
|                   21.35|
+------------------------+



2)	What is the distribution of “order_status”

In [19]:
df.groupBy(col("order_status")).\
    agg(count("order_status").alias("count")).\
        show()

+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|  983|
|    canceled|    3|
|     shipped|   12|
|  processing|    1|
|    invoiced|    1|
+------------+-----+



3)	How many unique states are there?

In [20]:
df.select(col("customer_state")).distinct().show()

+--------------+
|customer_state|
+--------------+
|            MS|
|            CE|
|            MG|
|            DF|
|            RO|
|            AM|
|            MT|
|            SP|
|            PB|
|            BA|
|            SE|
|            RJ|
|            AC|
|            PR|
|            RR|
|            TO|
|            ES|
|            AL|
|            RN|
|            SC|
+--------------+
only showing top 20 rows



In [21]:
state_count=df.select(col("customer_state")).distinct().count()

In [22]:
state_count

26

4)	Are there any missing values in the dataset?

In [25]:
df_flagged = df.withColumn("has_null", lit(False))
for col_name in df.columns:
    df_flagged = df_flagged.withColumn("has_null", col("has_null") | col(col_name).isNull())

rows_with_null = df_flagged.filter(col("has_null")==True)
rows_with_null.show()

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------------+--------------+------------------------+------------+--------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date|       customer_city|customer_state|customer_zip_code_prefix|review_score|has_null|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------------+--------------+------------------------+------------+--------+
|129|   delivered|               199.9|              15.15|              1|     2017-09-03 14:22:00|2017-09-03 14:30:00|                         NULL|      RIO DE JANEIRO|            RJ|                     212|           5|    true|
|155|   delivered|               14.03|              12.35|     

In [27]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- review_score: integer (nullable = true)



5)	Top 5 cities with most order

In [30]:
df_temp=df.groupBy(col("customer_city")).agg(count("*").alias("order_count")).orderBy(col("order_count").desc())

In [32]:
df_temp.show(5)

+--------------+-----------+
| customer_city|order_count|
+--------------+-----------+
|     Sao Paulo|        143|
|RIO DE JANEIRO|         74|
|      BRASILIA|         24|
|Belo Horizonte|         21|
|      Curitiba|         19|
+--------------+-----------+
only showing top 5 rows



6)	Percent of orders delivered/ canceled etc

In [36]:
delivered_count=df.select(col("order_status")).filter(col("order_status")=="delivered").count()
delivered_count
canceled_count=df.select(col("order_status")).filter(col("order_status")=="canceled").count()
canceled_count

3

In [38]:
total_count=df.select("*").count()
total_count

1000

In [40]:
delivered_pct=(delivered_count/total_count)*100
delivered_pct

98.3

In [41]:
canceled_pct=(canceled_count/total_count)*100
canceled_pct

0.3

Question set 2

1)	Calculate the Total sales in each customer city

In [42]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- review_score: integer (nullable = true)



In [44]:
df.groupBy(col("customer_city")).agg(round(sum(col("order_products_value")*col("order_items_qty")),2).alias("Total_Sales_by_city")).\
    show()

+--------------------+-------------------+
|       customer_city|Total_Sales_by_city|
+--------------------+-------------------+
|    Francisco Morato|              84.79|
|             Aracati|              102.0|
|           Americana|             675.78|
|                Tupa|               35.9|
|            Caieiras|              207.6|
|              Palmas|               99.6|
|             Maracas|             109.95|
|              Marica|              428.8|
|           Arapiraca|              349.9|
|     Pindamonhangaba|              294.8|
|        Santa Isabel|               19.9|
|         Sete Lagoas|             398.89|
|              Itajai|               88.0|
|              Guarai|               9.97|
|           Queimadas|               58.9|
|             Jaguare|               65.0|
|          Indaiatuba|             505.58|
|Sao Joaquim da Barra|               89.0|
|            Quintana|              349.9|
|                Ipua|              109.9|
+----------

2)	Correlation between order value order freight and item quantity

In [46]:
order_value_order_freight_corr = df.stat.corr("order_freight_value", "order_items_qty")
order_value_order_freight_corr

0.6330066852641087

3)	Calculate the Average Order delivery time / order approval time

In [47]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- review_score: integer (nullable = true)



Average delivery time

In [49]:
from pyspark.sql.functions import col, avg, unix_timestamp

# Filter out rows with null values in order_delivered_customer_date
filtered_df = df.filter(col("order_delivered_customer_date").isNotNull())

# Calculate the delivery time in seconds
delivery_time_df = filtered_df.withColumn(
    "delivery_time_seconds",
    unix_timestamp(col("order_delivered_customer_date")) - unix_timestamp(col("order_purchase_timestamp"))
)



# Calculate the average delivery time in seconds
average_delivery_time_seconds = delivery_time_df.select(avg(col("delivery_time_seconds"))).first()[0]

# Convert the average delivery time to days
average_delivery_time_days = average_delivery_time_seconds / (60 * 60 * 24)

print(f"Average Order Delivery Time: {average_delivery_time_days} days")

Average Order Delivery Time: 12.326998835491933 days


In [51]:
delivery_time_df.show()

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+---------------------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date|      customer_city|customer_state|customer_zip_code_prefix|review_score|delivery_time_seconds|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+---------------------+
|  1|   delivered|                79.0|               17.8|              1|     2017-10-02 10:56:00|2017-10-02 11:07:00|          2017-10-10 21:25:00|           Luziania|            GO|                     728|           5|               728940|
|  2|   delivere

In [54]:
delivery_time_df.select(col("delivery_time_seconds")).show()

+---------------------+
|delivery_time_seconds|
+---------------------+
|               728940|
|              1190760|
|               811680|
|              1141200|
|               248340|
|              1429200|
|               176580|
|               863100|
|               848340|
|              1574340|
|              1093020|
|               497160|
|              1043940|
|               425280|
|               984120|
|              1197060|
|              1492680|
|              1119300|
|               584700|
|              1146480|
+---------------------+
only showing top 20 rows



In [52]:
delivery_time_df.select(avg(col("delivery_time_seconds"))).show()

+--------------------------+
|avg(delivery_time_seconds)|
+--------------------------+
|         1065052.699386503|
+--------------------------+



In [53]:
delivery_time_df.select(avg(col("delivery_time_seconds"))).first()[0]

1065052.699386503

Average approval time

In [50]:
# Filter out rows with null values in order_aproved_at
filtered_df_approval = df.filter(col("order_aproved_at").isNotNull())

# Calculate the approval time in seconds
approval_time_df = filtered_df_approval.withColumn(
    "approval_time_seconds",
    unix_timestamp(col("order_aproved_at")) - unix_timestamp(col("order_purchase_timestamp"))
)

# Calculate the average approval time in seconds
average_approval_time_seconds = approval_time_df.select(avg(col("approval_time_seconds"))).first()[0]

# Convert the average approval time to hours
average_approval_time_hours = average_approval_time_seconds / (60 * 60)

print(f"Average Order Approval Time: {average_approval_time_hours} hours")


Average Order Approval Time: 10.507299999999999 hours


4)	Calculate the Average review score per Order

In [62]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- review_score: integer (nullable = true)



In [65]:
df.groupBy(col("order_status")) \
  .agg(round(avg(col("review_score")), 2).alias("avg_review_score")) \
  .show()

+------------+----------------+
|order_status|avg_review_score|
+------------+----------------+
|   delivered|            4.14|
|    canceled|             1.0|
|     shipped|            1.25|
|  processing|             1.0|
|    invoiced|             1.0|
+------------+----------------+



5)	Find the top 3 cities with fastest and slowest delivery times

In [68]:
# Filter out rows with null values in order_aproved_at
filtered_df_approval = df.filter(col("order_aproved_at").isNotNull())

# Calculate the approval time in seconds
approval_time_df = filtered_df_approval.withColumn(
    "approval_time_seconds",
    unix_timestamp(col("order_aproved_at")) - unix_timestamp(col("order_purchase_timestamp"))
)

top 3 slowest

In [69]:
delivery_time_df.orderBy(col("delivery_time_seconds").desc()).show(3)

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------+--------------+------------------------+------------+---------------------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date| customer_city|customer_state|customer_zip_code_prefix|review_score|delivery_time_seconds|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------+--------------+------------------------+------------+---------------------+
|253|   delivered|                29.0|               4.48|              1|     2018-02-01 07:47:00|2018-02-01 08:10:00|          2018-04-23 15:49:00|RIO DE JANEIRO|            RJ|                     220|           5|              7027320|
|659|   delivered|               149

top 3 fastest

In [70]:
delivery_time_df.orderBy(col("delivery_time_seconds")).show(3)

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------------+--------------+------------------------+------------+---------------------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date|       customer_city|customer_state|customer_zip_code_prefix|review_score|delivery_time_seconds|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+--------------------+--------------+------------------------+------------+---------------------+
| 45|   delivered|               69.49|              13.61|              1|     2018-06-04 16:44:00|2018-06-05 04:31:00|          2018-06-05 04:31:00|Sao Bernardo do C...|            SP|                      98|           4|                42420|
|396|   deli

6)	Relation between delivery time and review score (average delivery time for each review score) / Correlation between delivery time and review score

In [71]:
delivery_time_df.stat.corr("delivery_time_seconds","review_score")

0.021838753266314088

Save as table orders

In [72]:
df.write.saveAsTable("orders_table",header=True)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/07/21 05:21:22 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [74]:
spark.sql("""
          
          Select * from orders_table;
          
          """).show()

+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+
| Id|order_status|order_products_value|order_freight_value|order_items_qty|order_purchase_timestamp|   order_aproved_at|order_delivered_customer_date|      customer_city|customer_state|customer_zip_code_prefix|review_score|
+---+------------+--------------------+-------------------+---------------+------------------------+-------------------+-----------------------------+-------------------+--------------+------------------------+------------+
|  1|   delivered|                79.0|               17.8|              1|     2017-10-02 10:56:00|2017-10-02 11:07:00|          2017-10-10 21:25:00|           Luziania|            GO|                     728|           5|
|  2|   delivered|               119.9|              27.16|              1|     2018-07-24 20:41:00|2018

In [75]:
spark.stop()